código original: https://keras.io/examples/vision/knowledge_distillation/

# Destilação de conhecimento
Nesse jupyter iremos criar 2 modelos para classificação do mnist, um maix complexo que servirá como professor e um mais simples como estudante.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

Pelo fato que durante o treinamento do estudante precisar da inferencia do professor, isso faz como que mude a estratégia padrão de treinamento, por conta disso criaremos um modelo customizado herdando as caracteristicas do keras.Model e aplicando um override nos métodos de treinamento e validação.

In [2]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(self, optimizer, metrics, student_loss_fn, distillation_loss_fn, alpha=0.1, temperature=3):
        """ Configurações.

        Args:
            optimizer: otimizador para o treino do estudante
            metrics: métricas adcionais para avaliar o treinamento
            student_loss_fn: Loss function aplicada as predições do estudante x ground truth
            distillation_loss_fn: Loss function que irá levar em consideração a predição do professor
            alpha: peso para distribuir o fator de importancia entre o loss function do estudante e da destilação
            temperature: Suavização da distribuição.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):  # override
        x, y = data

        # training = false pois o professor já foi treinado e não queremos alterar seus pesos
        teacher_predictions = self.teacher(x, training=False)  # diferente de self.teacher.predict!

        with tf.GradientTape() as tape:
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss  # ponderação do peso

        # backpropagation
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # atualização dos pesos
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # atualiza as métricas
        self.compiled_metrics.update_state(y, student_predictions)

        # Returna um dict com todas métricas de performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data): #  override
        x, y = data

        # Gera predições
        y_prediction = self.student(x, training=False)

        # Calcula o loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Atualiza as métricas
        self.compiled_metrics.update_state(y, y_prediction)

        # Returna um dict com todas métricas de performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"val_student_loss": student_loss})
        return results

### Criação do modeslo

Arquitetura do professor ligeiramente mais complexa que do estudante

In [3]:
# Professor
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

In [4]:
# Estudante
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

Modelo clonado do estudante que iremos utilizar para validação do conceito de professor / estudante

In [5]:
student_scratch = keras.models.clone_model(student)  # clona o grafo do modelo

## Leitura do dataset

In [6]:
with np.load("./data/mnist.npz", allow_pickle=True) as f:
    x_train, y_train = f['x_train'][:10000], f['y_train'][:10000]
    x_test, y_test = f['x_test'][:10000], f['y_test'][:10000]

In [7]:
# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

## Treinamento

#### Treinamento do professor

In [8]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
125/313 [==========>...................] - ETA: 31s - loss: 0.8475 - sparse_categorical_accuracy: 0.7231 ETA: 32s - loss: 0.8761 - sparse_categorica

KeyboardInterrupt: 

#### Treinamento do estudante com ajuda do professor

In [ ]:
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

#### Treinamento do estudante sem ajuda do professor

In [ ]:
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)